In [1]:
#Todos os imports necessários para aplicação.

import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import create_map, struct, col


In [2]:
builder = SparkSession.builder.appName("QUALIS SPARK")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark

In [5]:
LANDED = './'
RAW = '../raw/'

In [6]:
file = spark.read.format('csv')\
    .option("header", "true")\
    .option("delimiter", ";")\
    .load(LANDED+"2013a2016-detalhe-prod-bibliografica.csv")\
    .select(\
            col("NM_PROGRAMA_IES").alias("CURSO"),\
            "NM_PROGRAMA_IES",\
            "SG_ENTIDADE_ENSINO",\
            "AN_BASE_PRODUCAO",\
            "ID_ADD_PRODUCAO_INTELECTUAL"\
            , "DS_ISSN")\
    .filter("NM_PROGRAMA_IES like '%COMPUTAÇÃO%'")

In [8]:
file.toPandas().head()

,CURSO,NM_PROGRAMA_IES,SG_ENTIDADE_ENSINO,AN_BASE_PRODUCAO,ID_ADD_PRODUCAO_INTELECTUAL,DS_ISSN
0,SISTEMAS E COMPUTAÇÃO,SISTEMAS E COMPUTAÇÃO,UNIFACS,2013,3344592,INTERNATIONAL JOURNAL OF COMPUTER NETWORKS AND...
1,SISTEMAS E COMPUTAÇÃO,SISTEMAS E COMPUTAÇÃO,UNIFACS,2013,3344594,CYBER JOURNALS: MULTIDISCIPLINARY JOURNALS IN ...
2,SISTEMAS E COMPUTAÇÃO,SISTEMAS E COMPUTAÇÃO,UNIFACS,2013,3344595,COMPUTER TECHNOLOGY AND APPLICATION
3,SISTEMAS E COMPUTAÇÃO,SISTEMAS E COMPUTAÇÃO,UNIFACS,2013,3344598,SBC JOURNAL ON 3D INTERACTIVE SYSTEMS
4,SISTEMAS E COMPUTAÇÃO,SISTEMAS E COMPUTAÇÃO,UNIFACS,2013,3344599,ENGENHARIA DE SOFTWARE MAGAZINE


In [9]:
file.write.mode("overwrite").parquet(LANDED+"detalhe-curso-issn")

In [17]:
print("CSV")
print(file.count())
file_detalhe = spark.read.parquet(LANDED+"detalhe-curso-issn")
print("PARQUET")
print(file_detalhe.count())


CSV
7760
PARQUET
7760


In [18]:
file_qualis = spark.read.format('csv')\
    .option("header", "true")\
    .option("delimiter", ";")\
    .load(LANDED+"qualis.csv")\
    .select(\
            col("ISSN"),\
            col("Titulo").alias("TITULO"),\
            col("Area de Avaliacao").alias("AREA"),\
            col("Estrato").alias("QUALIS"))\
    .filter("AREA like '%COMPUTAÇÃO%'")
file_qualis.toPandas().head()

,ISSN,TITULO,AREA,QUALIS
0,1619-4500,4OR (BERLIN),CIÊNCIA DA COMPUTAÇÃO ...,B1
1,0094-6354,A.A.N.A. JOURNAL,CIÊNCIA DA COMPUTAÇÃO ...,B5
2,1528-7106,AACE INTERNATIONAL TRANSACTIONS,CIÊNCIA DA COMPUTAÇÃO ...,C
3,2212-6716,AASRI PROCEDIA,CIÊNCIA DA COMPUTAÇÃO ...,C
4,2316-9451,ABAKÓS,CIÊNCIA DA COMPUTAÇÃO ...,B5


In [25]:
join_qualis_curso = file\
    .join(file_qualis,\
          file.DS_ISSN == file_qualis.TITULO, how='inner')\
    .filter("DS_ISSN is not null AND QUALIS IS NOT NULL")
join_qualis_curso.toPandas().head()

,CURSO,NM_PROGRAMA_IES,SG_ENTIDADE_ENSINO,AN_BASE_PRODUCAO,ID_ADD_PRODUCAO_INTELECTUAL,DS_ISSN,ISSN,TITULO,AREA,QUALIS
0,ENGENHARIA DE SISTEMAS E COMPUTAÇÃO,ENGENHARIA DE SISTEMAS E COMPUTAÇÃO,UFRJ,2014,18697655,OPTIMIZATION METHODS & SOFTWARE (PRINT),1055-6788,OPTIMIZATION METHODS & SOFTWARE (PRINT),CIÊNCIA DA COMPUTAÇÃO ...,A2
1,ENGENHARIA DE SISTEMAS E COMPUTAÇÃO,ENGENHARIA DE SISTEMAS E COMPUTAÇÃO,UFRJ,2014,18698080,OPTIMIZATION METHODS & SOFTWARE (PRINT),1055-6788,OPTIMIZATION METHODS & SOFTWARE (PRINT),CIÊNCIA DA COMPUTAÇÃO ...,A2
2,CIÊNCIAS DA COMPUTAÇÃO E MATEMÁTICA COMPUTACIONAL,CIÊNCIAS DA COMPUTAÇÃO E MATEMÁTICA COMPUTACIONAL,USP/SC,2016,21065300,OPTIMIZATION METHODS & SOFTWARE (PRINT),1055-6788,OPTIMIZATION METHODS & SOFTWARE (PRINT),CIÊNCIA DA COMPUTAÇÃO ...,A2
3,CIÊNCIAS DA COMPUTAÇÃO,CIÊNCIAS DA COMPUTAÇÃO,UFSC,2013,18816952,REAL-TIME SYSTEMS,0922-6443,REAL-TIME SYSTEMS,CIÊNCIA DA COMPUTAÇÃO ...,B1
4,CIÊNCIA DA COMPUTAÇÃO - UFBA - UNIFACS - UEFS,CIÊNCIA DA COMPUTAÇÃO - UFBA - UNIFACS - UEFS,UFBA,2016,21083101,REAL-TIME SYSTEMS,0922-6443,REAL-TIME SYSTEMS,CIÊNCIA DA COMPUTAÇÃO ...,B1


In [27]:
join_qualis_curso.write.mode("overwrite").parquet(RAW+"producao_qualis_computacao")